# ML match for LOFAR and the combined PanSTARRS WISE catalogue: Get all sources above the threshold

In this notebook the maximum likelihood cross-match between the LOFAR HETDEX catalogue and the combined PansSTARRS WISE catalogue is computed. However, we take all the sources that have likelihood ratios above the threshold found in the first run.

## Configuration

### Load libraries and setup

In [1]:
import numpy as np
from astropy.table import Table, join
from astropy import units as u
from astropy.coordinates import SkyCoord, search_around_sky
from IPython.display import clear_output
import pickle
import os

In [2]:
from mltier1 import (get_center, get_n_m, estimate_q_m, Field, SingleMLEstimator, MultiMLEstimator,
                     parallel_process, get_sigma, get_q_m)

In [3]:
%load_ext autoreload

In [4]:
%autoreload

In [5]:
from IPython.display import clear_output

In [6]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### General configuration

In [7]:
save_intermediate = True
plot_intermediate = True

In [8]:
idp = "idata/final_abv_extended"

In [9]:
if not os.path.isdir(idp):
    os.makedirs(idp)

### Area limits

In [10]:
# Busy week Edinburgh 2017
ra_down = 172.09
ra_up = 187.5833
dec_down = 46.106
dec_up = 56.1611

In [11]:
# Busy week Hatfield 2017
ra_down = 170.
ra_up = 190.
dec_down = 46.8
dec_up = 55.9

In [12]:
# Full field July 2017
ra_down = 160.
ra_up = 232.
dec_down = 42.
dec_up = 62.

In [13]:
field = Field(170.0, 190.0, 46.8, 55.9)

In [14]:
field_full = Field(160.0, 232.0, 42.0, 62.0)

## Load data

In [15]:
combined_all = Table.read("pw.fits")

In [16]:
lofar_all = Table.read("data/LOFAR_HBA_T1_DR1_catalog_v0.9.srl.fits")

In [17]:
np.array(combined_all.colnames)

array(['AllWISE', 'objID', 'ra', 'dec', 'raErr', 'decErr', 'W1mag',
       'W1magErr', 'i', 'iErr'],
      dtype='<U8')

In [18]:
np.array(lofar_all.colnames)

array(['Source_Name', 'RA', 'E_RA', 'E_RA_tot', 'DEC', 'E_DEC',
       'E_DEC_tot', 'Peak_flux', 'E_Peak_flux', 'E_Peak_flux_tot',
       'Total_flux', 'E_Total_flux', 'E_Total_flux_tot', 'Maj', 'E_Maj',
       'Min', 'E_Min', 'PA', 'E_PA', 'Isl_rms', 'S_Code', 'Mosaic_ID',
       'Isl_id'],
      dtype='<U16')

### Filter catalogues

In [19]:
lofar = field_full.filter_catalogue(lofar_all[lofar_all['Maj'] >= 30.], 
                                         colnames=("RA", "DEC"))

In [20]:
combined = field_full.filter_catalogue(combined_all, 
                               colnames=("ra", "dec"))

### Additional data

In [21]:
combined["colour"] = combined["i"] - combined["W1mag"]

In [22]:
combined_aux_index = np.arange(len(combined))

### Sky coordinates

In [23]:
coords_combined = SkyCoord(combined['ra'], 
                           combined['dec'], 
                           unit=(u.deg, u.deg), 
                           frame='icrs')

In [24]:
coords_lofar = SkyCoord(lofar['RA'], 
                       lofar['DEC'], 
                       unit=(u.deg, u.deg), 
                       frame='icrs')

### Class of sources in the combined catalogue

The sources are grouped depending on the available photometric data.

In [25]:
combined_matched = (~np.isnan(combined["i"]) & ~np.isnan(combined["W1mag"])) # Matched i-W1 sources

In [26]:
combined_panstarrs = (~np.isnan(combined["i"]) & np.isnan(combined["W1mag"])) # Sources with only i-band

In [27]:
combined_wise =(np.isnan(combined["i"]) & ~np.isnan(combined["W1mag"])) # Sources with only W1-band

In [28]:
combined_i = combined_matched | combined_panstarrs
combined_w1 = combined_matched | combined_wise
#combined_only_i = combined_panstarrs & ~combined_matched
#combined_only_w1 = combined_wise & ~combined_matched

In [29]:
print("Total    - ", len(combined))
print("i and W1 - ", np.sum(combined_matched))
print("Only i   - ", np.sum(combined_panstarrs))
print("With i   - ", np.sum(combined_i))
print("Only W1  - ", np.sum(combined_wise))
print("With W1  - ", np.sum(combined_w1))

Total    -  31803022
i and W1 -  8196213
Only i   -  18583323
With i   -  26779536
Only W1  -  5023475
With W1  -  13219688


### Colour categories

The colour categories will be used after the first ML match

In [30]:
colour_limits = [-0.5, 0.0, 0.5, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.5, 4.0]

In [31]:
# Start with the W1-only, i-only and "less than lower colour" bins
colour_bin_def = [{"name":"only W1", "condition": combined_wise},
                  {"name":"only i", "condition": combined_panstarrs},
                  {"name":"-inf to {}".format(colour_limits[0]), 
                   "condition": (combined["colour"] < colour_limits[0])}]

# Get the colour bins
for i in range(len(colour_limits)-1):
    name = "{} to {}".format(colour_limits[i], colour_limits[i+1])
    condition = ((combined["colour"] >= colour_limits[i]) & 
                 (combined["colour"] < colour_limits[i+1]))
    colour_bin_def.append({"name":name, "condition":condition})

# Add the "more than higher colour" bin
colour_bin_def.append({"name":"{} to inf".format(colour_limits[-1]), 
                       "condition": (combined["colour"] >= colour_limits[-1])})

/disk2/jsm/prog/anaconda/envs/py36/lib/python3.6/site-packages/astropy/table/column.py:928: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, oper)(other)
/disk2/jsm/prog/anaconda/envs/py36/lib/python3.6/site-packages/astropy/table/column.py:928: RuntimeWarning: invalid value encountered in greater_equal
  return getattr(self.data, oper)(other)


In [32]:
combined["category"] = np.nan
for i in range(len(colour_bin_def)):
    combined["category"][colour_bin_def[i]["condition"]] = i

In [33]:
np.sum(np.isnan(combined["category"]))

11

We get the number of sources of the combined catalogue in each colour category. It will be used at a later stage to compute the $Q_0$ values

In [34]:
numbers_combined_bins = np.array([np.sum(a["condition"]) for a in colour_bin_def])

In [35]:
numbers_combined_bins

array([ 5023475, 18583323,   617057,   435585,   680625,   914717,
         656883,   775440,   830399,   803840,   712001,   578606,
         437076,   307929,   317143,    94996,    33916])

## Maximum Likelihood

In [36]:
bin_list, centers, Q_0_colour, n_m, q_m = pickle.load(open("lofar_params.pckl", "rb"))

In [37]:
likelihood_ratio_function = MultiMLEstimator(Q_0_colour, n_m, q_m, centers)

### ML match

In [38]:
radius = 15

In [39]:
lr_threshold = 0.36

In [40]:
selection = ~np.isnan(combined["category"]) # Avoid the dreaded sources with no actual data
catalogue = combined[selection]

In [41]:
def apply_ml(i, likelihood_ratio_function):
    idx_0 = idx_i[idx_lofar == i]
    d2d_0 = d2d[idx_lofar == i]
    
    category = catalogue["category"][idx_0].astype(int)
    mag = catalogue["i"][idx_0]
    mag[category == 0] = catalogue["W1mag"][idx_0][category == 0]
    
    lofar_ra = lofar[i]["RA"]
    lofar_dec = lofar[i]["DEC"]
    lofar_pa = lofar[i]["PA"]
    lofar_maj_err = lofar[i]["E_Maj"]
    lofar_min_err = lofar[i]["E_Min"]
    c_ra = catalogue["ra"][idx_0]
    c_dec = catalogue["dec"][idx_0]
    c_ra_err = catalogue["raErr"][idx_0]
    c_dec_err = catalogue["decErr"][idx_0]
    
    sigma = get_sigma(lofar_maj_err, lofar_min_err, lofar_pa, 
                      lofar_ra, lofar_dec, 
                      c_ra, c_dec, c_ra_err, c_dec_err)

    lr_0 = likelihood_ratio_function(mag, d2d_0.arcsec, sigma, category)
    
    chosen_index = (lr_0 >= lr_threshold)
    result = [combined_aux_index[selection][idx_0[chosen_index]], # Index
              (d2d_0.arcsec)[chosen_index],                        # distance
              lr_0[chosen_index]]                                  # LR
    return result

### Run the cross-match

In [42]:
idx_lofar, idx_i, d2d, d3d = search_around_sky(
    coords_lofar, coords_combined[selection], radius*u.arcsec)

In [43]:
idx_lofar_unique = np.unique(idx_lofar)

In [44]:
total_sources = len(idx_lofar_unique)

### Run the ML matching

In [45]:
def ml(i):
    return apply_ml(i, likelihood_ratio_function)

In [46]:
res = parallel_process(idx_lofar_unique, ml, n_jobs=8)

100%|██████████| 4.30K/4.30K [02:17<00:00, 31.2it/s]
100%|██████████| 4300/4300 [00:00<00:00, 287020.50it/s]


### Selection and match

In [47]:
lofar_aux_index = np.arange(len(lofar))

In [48]:
lr = []
lr_dist = []
lr_index = []
lr_order = []
lr_lofar_index = []

for i, idx in enumerate(idx_lofar_unique):
    result = res[i]
    n = len(result[0])
    lofar_index = lofar_aux_index[idx]
    if n > 0:
        order = np.argsort(result[2])[::-1]
        lr.extend(result[2][order])
        lr_dist.extend(result[1][order])
        lr_index.extend(result[0][order])
        lr_order.extend(np.arange(n, dtype=int) + 1)
        lr_lofar_index.extend(np.ones(n, dtype=int)*lofar_index)
    else:
        lr.append(np.nan)
        lr_dist.append(np.nan)
        lr_index.append(np.nan)
        lr_order.append(np.nan)
        lr_lofar_index.append(lofar_index)

In [49]:
aux_table = Table()
aux_table['aux_index'] = lr_lofar_index
aux_table['lr'] = lr
aux_table['lr_dist'] = lr_dist
aux_table['lr_index'] = lr_index
aux_table['lr_order'] = lr_order

In [50]:
aux_table

aux_index,lr,lr_dist,lr_index,lr_order
int64,float64,float64,float64,float64
0,4.15630601591,7.21952240748,11408661.0,1.0
1,556.592596532,1.14982263683,10930603.0,1.0
1,2.1454448408,1.14982263683,10930604.0,2.0
2,nan,nan,nan,nan
3,0.781141027982,1.2929719743,10915215.0,1.0
3,0.519191191864,2.43528867037,10916198.0,2.0
4,nan,nan,nan,nan
5,nan,nan,nan,nan
6,nan,nan,nan,nan


In [51]:
lofar["aux_index"] = lofar_aux_index

In [52]:
lofar_lr = join(lofar, aux_table, join_type='outer', keys='aux_index')

In [53]:
lofar_lr

Source_Name,RA,E_RA,E_RA_tot,DEC,E_DEC,E_DEC_tot,Peak_flux,E_Peak_flux,E_Peak_flux_tot,Total_flux,E_Total_flux,E_Total_flux_tot,Maj,E_Maj,Min,E_Min,PA,E_PA,Isl_rms,S_Code,Mosaic_ID,Isl_id,aux_index,lr,lr_dist,lr_index,lr_order
,deg,arcsec,arcsec,deg,arcsec,arcsec,mJy / beam,mJy / beam,mJy / beam,mJy,mJy,mJy,arcsec,arcsec,arcsec,arcsec,deg,deg,mJy / beam,,,,,,,,
str24,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,str8,int32,int64,float64,float64,float64,float64
ILTJ122034.036+503026.19,185.14181653,1.6803150482,1.68485208756,50.5072751983,2.03186813265,2.03562177337,0.409341156683,0.0656106387635,0.104915028576,6.56610340421,0.0695943200116,1.31506346842,33.6151577179,5.61845082329,17.1843882644,2.64137679093,37.9521756126,18.3834711277,0.103788341221,S,P22Hetde,387,0,4.15630601591,7.21952240748,11408661.0,1.0
ILTJ122020.649+493520.07,185.08603568,0.814015307542,0.823340037081,49.5889084893,0.759973787986,0.769953280514,0.507827290379,0.0372784310345,0.108190682106,7.2317431976,0.039818665574,1.4468966491,30.6081876366,2.34172343559,16.7531254717,1.17999733002,49.7651346205,9.61065545882,0.0560113621759,S,P22Hetde,399,1,556.592596532,1.14982263683,10930603.0,1.0
ILTJ122020.649+493520.07,185.08603568,0.814015307542,0.823340037081,49.5889084893,0.759973787986,0.769953280514,0.507827290379,0.0372784310345,0.108190682106,7.2317431976,0.039818665574,1.4468966491,30.6081876366,2.34172343559,16.7531254717,1.17999733002,49.7651346205,9.61065545882,0.0560113621759,S,P22Hetde,399,1,2.1454448408,1.14982263683,10930604.0,2.0
ILTJ121913.652+492142.88,184.806882892,1.03526505386,1.04261288478,49.3619120787,1.5199507989,1.52496502479,0.32265640886,0.0366216077033,0.0741985746353,5.74680165471,0.0386255622924,1.15000917579,31.9249689259,3.71656914119,20.0882863707,2.22292789957,20.8417352622,16.0887693019,0.0597976068093,S,P22Hetde,759,2,nan,nan,nan,nan
ILTJ121910.749+492049.61,184.794788594,1.41814290396,1.42351578559,49.3471151607,1.36333133491,1.36891936376,0.313950921247,0.0305315967326,0.0698196651175,8.57330231112,0.0316306547195,1.71495218564,40.4729460999,4.00995476772,24.2945414801,2.31904707592,48.4957546418,13.3287306941,0.0599399863859,S,P22Hetde,775,3,0.781141027982,1.2929719743,10915215.0,1.0
ILTJ121910.749+492049.61,184.794788594,1.41814290396,1.42351578559,49.3471151607,1.36333133491,1.36891936376,0.313950921247,0.0305315967326,0.0698196651175,8.57330231112,0.0316306547195,1.71495218564,40.4729460999,4.00995476772,24.2945414801,2.31904707592,48.4957546418,13.3287306941,0.0599399863859,S,P22Hetde,775,3,0.519191191864,2.43528867037,10916198.0,2.0
ILTJ121856.362+502630.32,184.734842009,0.0349134889266,0.128401119377,50.4417576778,0.0959182676008,0.156423175418,267.347898652,0.697937677614,53.4741346219,1386.08003266,2.53004016267,277.227551626,42.5142169085,0.233953233891,12.2043960762,0.0551594224043,75.6567667866,0.399159737233,0.697937677614,M,P22Hetde,904,4,nan,nan,nan,nan
ILTJ121839.336+502549.84,184.663900816,0.0453994495482,0.131639681584,50.4305116771,0.0931867086809,0.154763233435,525.156024562,0.703760422766,105.033562655,1463.23476862,2.49456978375,292.657585588,32.4963076544,0.233572536285,11.9460899144,0.0708949645617,70.0831786146,0.556076324599,0.703760422766,M,P22Hetde,989,5,nan,nan,nan,nan


## Save combined catalogue

In [54]:
combined["lr_index"] = combined_aux_index.astype(float)

In [55]:
for col in ['lr', 'lr_dist', 'lr_index', 'lr_order']:
    lofar_lr[col].fill_value = np.nan

In [56]:
pwl = join(lofar_lr.filled(), combined, join_type='left', keys='lr_index')

In [57]:
len(pwl)

5454

In [58]:
pwl_columns = pwl.colnames

In [59]:
for col in pwl_columns:
    fv = pwl[col].fill_value
    #print(col, fv)
    if (isinstance(fv, np.float64) and (fv != 1e+20)):
        print(col, fv)
        pwl[col].fill_value = 1e+20

dec 1.0
W1mag 1.0
i 1.0
colour 1.0
category 1.0


In [60]:
columns_save = ['Source_Name', 'RA', 'E_RA', 'E_RA_tot', 'DEC', 'E_DEC', 'E_DEC_tot',
 'Peak_flux', 'E_Peak_flux', 'E_Peak_flux_tot', 'Total_flux', 'E_Total_flux', 'E_Total_flux_tot',
 'Maj', 'E_Maj', 'Min', 'E_Min', 'PA', 'E_PA', 'Isl_rms', 'S_Code', 'Mosaic_ID', 'Isl_id',
 'AllWISE', 'objID', 'ra', 'dec', 'raErr', 'decErr',
 'W1mag', 'W1magErr', 'i', 'iErr', 'colour', 'category',
 'lr', 'lr_dist', 'lr_order']

In [61]:
pwl[columns_save].write('lofar_multi_lr_pw-extended.fits', format="fits")